[Reference](https://medium.com/@life-is-short-so-enjoy-it/fastapi-experiment-middleware-feature-c0a0c7314d74)

# Experiment 1: Build a simple middleware

In [2]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.3 MB/s eta 0:00:00


In [3]:
import time
from fastapi import FastAPI, Request

app = FastAPI()

@app.get("/")
async def root():
    return "Wonderful!!"

In [4]:
# !curl -i 127.0.0.1:8000

In [5]:
import time
from fastapi import FastAPI, Request

app = FastAPI()

# Implemented and added custom middleware to FastAPI
@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    response.headers["X-Process-Time"] = str(time.time() - start_time)
    return response

@app.get("/")
async def root():
    return "Wonderful!!"

# Experiment 2: What if there is no matched path? Will the middleware be executed?

In [6]:
import time
from fastapi import FastAPI, Request

app = FastAPI()

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    print("in add_process_time_header middleware.") # dummy message
    start_time = time.time()
    response = await call_next(request)
    response.headers["X-Process-Time"] = str(time.time() - start_time)
    return response

@app.get("/")
async def root():
    return "Wonderful!!"

# Experiment 3: Does the middleware work even for the non-async endpoint?


In [7]:
import time
from fastapi import FastAPI, Request

app = FastAPI()

@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    response.headers["X-Process-Time"] = str(time.time() - start_time)
    return response

@app.get("/")
def root():
    return "Wonderful!! - Sync"

# Experiment 4: What is the sequence of executions?

In [9]:
async def add_process_time_header(request: Request, call_next):
    # 1. Do thing before the matched path operation
    start_time = time.time()
    # 2. find / execute the matched path operation
    response = await call_next(request)
    # 3. Do thing after the matched path operation
    response.headers["X-Process-Time"] = str(time.time() - start_time)
    return response

In [10]:
# ref: https://github.com/encode/starlette/blob/master/starlette/applications.py

def build_middleware_stack(self) -> ASGIApp:
    debug = self.debug
    error_handler = None
    exception_handlers: typing.Dict[
        typing.Any, typing.Callable[[Request, Exception], Response]
    ] = {}

    for key, value in self.exception_handlers.items():
        if key in (500, Exception):
            error_handler = value
        else:
            exception_handlers[key] = value

    middleware = (
        [Middleware(ServerErrorMiddleware, handler=error_handler, debug=debug)]
        + self.user_middleware
        + [
            Middleware(
                ExceptionMiddleware, handlers=exception_handlers, debug=debug
            )
        ]
    )

    app = self.router
    for cls, options in reversed(middleware):
        app = cls(app=app, **options)
    return app

# Experiment 5: ASGI Middlewares

In [12]:
# ref: https://github.com/encode/starlette/blob/master/starlette/middleware/gzip.py
class GZipMiddleware:
    def __init__(
        self, app: ASGIApp, minimum_size: int = 500, compresslevel: int = 9
    ) -> None:
        self.app = app
        self.minimum_size = minimum_size
        self.compresslevel = compresslevel

    async def __call__(self, scope: Scope, receive: Receive, send: Send) -> None:
        if scope["type"] == "http":
            headers = Headers(scope=scope)
            if "gzip" in headers.get("Accept-Encoding", ""):
                responder = GZipResponder(
                    self.app, self.minimum_size, compresslevel=self.compresslevel
                )
                await responder(scope, receive, send)
                return
        await self.app(scope, receive, send)

In [13]:
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Experiment 5: How is the custom middleware added?

In [14]:
@app.middleware("http")
async def add_process_time_header(request: Request, call_next):
    start_time = time.time()
    response = await call_next(request)
    response.headers["X-Process-Time"] = str(time.time() - start_time)
    return response

In [15]:
# ref: https://github.com/tiangolo/fastapi/blob/master/fastapi/applications.py

def middleware(
    self, middleware_type: str
) -> Callable[[DecoratedCallable], DecoratedCallable]:
    def decorator(func: DecoratedCallable) -> DecoratedCallable:
        # this add_middleware is in Starlette
        # This is the format of adding ASGI middleware as you saw before.
        # BaseHTTPMiddleware is defined in Starlette
        # ref: https://github.com/encode/starlette/blob/master/starlette/middleware/base.py
        self.add_middleware(BaseHTTPMiddleware, dispatch=func)
        return func

    return decorator

In [16]:
# ref: https://github.com/encode/starlette/blob/master/starlette/applications.py

def add_middleware(self, middleware_class: type, **options: typing.Any) -> None:
    if self.middleware_stack is not None:  # pragma: no cover
        raise RuntimeError("Cannot add middleware after an application has started")
    self.user_middleware.insert(0, Middleware(middleware_class, **options))